# House Price Prediction - Step 2: Models

First, run the data cleaning notebook to prepare the dataset.

In [ ]:
# Run the data cleaning notebook
%run house_price_prediction_step1.ipynb

In [ ]:
import os
import sys
import time
import warnings

import numpy as np
import pandas as pd
import patsy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")

In [ ]:
current_path = os.getcwd()
assignment_dir = os.path.dirname(current_path) if "code" in current_path else current_path

data_out = os.path.join(assignment_dir, "data", "clean/")
output = os.path.join(assignment_dir, "output/")

import py_helper_functions as da

sns.set_theme(style="whitegrid")

In [ ]:
data = pd.read_csv(data_out + "airbnb_madrid_25q1_clean.csv")
print(f"Data shape: {data.shape}")

In [ ]:
# Basic property characteristics - core drivers of price
# NOTE: Neighbourhood excluded for cross-city validity
basic_lev = (
    "n_accommodates",      # Capacity is primary price driver
    "n_beds",              # Number of beds
    "f_property_type",     # Apartment vs House
    "f_room_type",         # Entire place vs Private vs Shared
    "n_days_since",        # Time on platform (experience proxy)
    "flag_days_since",     # Flag for imputed values
)

# Additional property features
basic_add = (
    "f_bathroom",          # Bathroom categories
    "n_bedrooms",          # Number of bedrooms
)

# Review information - quality signals
reviews = (
    "f_number_of_reviews",       # Review count categories
    "n_review_scores_rating",    # Rating score
    "flag_review_scores_rating", # Flag for imputed ratings
)

# Polynomial terms
poly_lev = (
    "n_accommodates2",     # Squared term for accommodates
    "ln_accommodates",     # Log transformation
)

# Host characteristics
host_vars = (
    "d_host_is_superhost",       # Superhost status
    "d_instant_bookable",        # Convenience feature
)

# Amenities
amenities = tuple([c for c in data.columns if c.startswith('d_') and 'host' not in c and 'instant' not in c])
print(f"Number of amenity variables: {len(amenities)}")

In [ ]:
# Price by room type and property type
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

data.groupby(['f_room_type', 'f_property_type'])['price'].mean().unstack().plot(
    kind='bar', ax=axes[0], rot=45
)
axes[0].set_title('Price by Room Type and Property Type')
axes[0].set_ylabel('Mean Price (€)')
axes[0].legend(title='Property Type')

data.groupby(['f_room_type', pd.cut(data['n_accommodates'], bins=[0,2,4,6,20])])['price'].mean().unstack().plot(
    kind='bar', ax=axes[1], rot=45
)
axes[1].set_title('Price by Room Type and Capacity')
axes[1].set_ylabel('Mean Price (€)')
axes[1].legend(title='Accommodates')

plt.tight_layout()
plt.savefig(output + 'interactions_plot.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# X1: Core interactions
X1 = (
    "f_room_type*f_property_type",
    "n_accommodates*f_room_type",
)

In [ ]:
modellev1 = "~ n_accommodates"
modellev2 = "~ " + " + ".join(basic_lev)
modellev3 = "~ " + " + ".join(basic_lev + basic_add)
modellev4 = "~ " + " + ".join(basic_lev + basic_add + reviews)
modellev5 = "~ " + " + ".join(basic_lev + basic_add + reviews + poly_lev)
modellev6 = "~ " + " + ".join(basic_lev + basic_add + reviews + poly_lev + host_vars + X1)
modellev7 = "~ " + " + ".join(basic_lev + basic_add + reviews + poly_lev + host_vars + X1 + amenities)

model_equations = [modellev1, modellev2, modellev3, modellev4, modellev5, modellev6, modellev7]
model_names = ['M1: Accommodates only', 'M2: Basic', 'M3: +Property features', 
               'M4: +Reviews', 'M5: +Polynomials', 'M6: +Host & Interactions', 'M7: +Amenities']

In [ ]:
np.random.seed(42)
data_work, data_holdout = train_test_split(data, test_size=0.2, random_state=42)
print(f"Training set: {data_work.shape[0]} rows")
print(f"Holdout set: {data_holdout.shape[0]} rows")

In [ ]:
n_folds = 5

print("\n" + "="*60)
print("RUNNING OLS MODELS WITH 5-FOLD CROSS-VALIDATION")
print("="*60)

cv_results = []
ols_times = []

for i, model in enumerate(model_equations):
    start_time = time.time()
    cv_result = da.ols_crossvalidator("price " + model, data_work, n_folds)
    elapsed_time = time.time() - start_time
    ols_times.append(elapsed_time)
    
    cv_result['Model'] = f'M{i+1}_OLS'
    cv_result['Time (s)'] = round(elapsed_time, 2)
    cv_results.append(cv_result)
    
    print(f"M{i+1}_OLS: Test RMSE = {cv_result['Test RMSE']:.2f}, Time = {elapsed_time:.2f}s")

ols_comparison = (
    pd.DataFrame(cv_results)
    .round(2)
    .assign(
        BIC=lambda x: x['BIC'].astype(int),
        Coefficients=lambda x: x['Coefficients'].astype(int),
    )
    [['Model', 'Coefficients', 'R-squared', 'BIC', 'Training RMSE', 'Test RMSE', 'Time (s)']]
)

print("\n--- OLS Model Comparison ---")
print(ols_comparison.to_string(index=False))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

df_melted = ols_comparison.melt(
    id_vars='Coefficients', 
    value_vars=['Test RMSE', 'Training RMSE']
)

sns.lineplot(data=df_melted, x='Coefficients', y='value', hue='variable', 
             marker='o', linewidth=2)
plt.xlabel('Number of Coefficients')
plt.ylabel('RMSE (€)')
plt.title('OLS Model Complexity vs. RMSE')
plt.legend(title='')
plt.savefig(output + 'ols_model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Use the most complex model specification for LASSO
vars_for_lasso = modellev7

# Create design matrix using patsy
y, X = patsy.dmatrices("price " + vars_for_lasso, data_work)
X_featnames = X.design_info.column_names

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Define range of lambda values
lambdas = np.arange(0.05, 1.01, 0.05)

# Run LASSO with 5-fold CV
start_time = time.time()
lasso_fit = LassoCV(alphas=lambdas, cv=5, random_state=42).fit(X, y)
lasso_time = time.time() - start_time

print(f"Optimal lambda: {lasso_fit.alpha_}")

from sklearn.metrics import r2_score, mean_squared_error

y_pred_train = lasso_fit.predict(X)
lasso_train_rmse = np.sqrt(mean_squared_error(y, y_pred_train))
lasso_r2 = r2_score(y, y_pred_train)

n = len(y)
k = np.sum(lasso_fit.coef_ != 0)
mse = mean_squared_error(y, y_pred_train)
lasso_bic = n * np.log(mse) + k * np.log(n)

print(f"LASSO Training RMSE: {lasso_train_rmse:.2f}")
print(f"LASSO R-squared: {lasso_r2:.3f}")
print(f"LASSO BIC: {lasso_bic:.0f}")

In [ ]:
rmse_by_lambda = (
    pd.DataFrame({
        'lambda': lasso_fit.alphas_,
        'Test RMSE': np.sqrt(lasso_fit.mse_path_).mean(axis=1)
    })
    .sort_values('lambda')
)

fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(rmse_by_lambda['lambda'], rmse_by_lambda['Test RMSE'], 'b-', linewidth=2)
plt.axvline(x=lasso_fit.alpha_, color='r', linestyle='--', label=f'Optimal λ = {lasso_fit.alpha_:.3f}')
plt.xlabel('Lambda (Regularization Parameter)')
plt.ylabel('Test RMSE (€)')
plt.title('LASSO: Cross-Validation RMSE by Lambda')
plt.legend()
plt.savefig(output + 'lasso_lambda_selection.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
lasso_coefs = pd.DataFrame({
    'Feature': X_featnames,
    'Coefficient': lasso_fit.coef_
})

nonzero_coefs = lasso_coefs[np.abs(lasso_coefs['Coefficient']) > 0.001].copy()
nonzero_coefs['Abs_Coef'] = np.abs(nonzero_coefs['Coefficient'])
nonzero_coefs = nonzero_coefs.sort_values('Abs_Coef', ascending=False)

print(f"\nLASSO selected {len(nonzero_coefs)} features out of {len(X_featnames)}")
print("\nTop 20 features by absolute coefficient:")
print(nonzero_coefs.head(20).to_string(index=False))

In [ ]:
lasso_rmse = rmse_by_lambda.loc[rmse_by_lambda['lambda'] == lasso_fit.alpha_, 'Test RMSE'].values[0]

lasso_row = {
    'Model': 'LASSO',
    'Coefficients': np.sum(lasso_fit.coef_ != 0),
    'R-squared': round(lasso_r2, 2),
    'BIC': int(lasso_bic),
    'Training RMSE': round(lasso_train_rmse, 2),
    'Test RMSE': round(lasso_rmse, 2),
    'Time (s)': round(lasso_time, 2)
}
comparison = pd.concat([ols_comparison, pd.DataFrame([lasso_row])], ignore_index=True)

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Predictors for tree models
predictors = basic_lev + basic_add + reviews + poly_lev + host_vars + amenities

# Create design matrices
y_train, X_train = patsy.dmatrices("price ~ " + " + ".join(predictors), data_work)
y_train = np.ravel(y_train)
X_featnames = X_train.design_info.column_names

print(f"Training data: {X_train.shape[0]} obs, {X_train.shape[1]} features")

In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

rf_tune_grid = {
    "max_features": [5, 8, 10, 12],
    "min_samples_leaf": [5, 10, 20]
}

rf_cv = GridSearchCV(
    estimator=rf,
    param_grid=rf_tune_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=1,
    n_jobs=1
)

In [ ]:
start_time = time.time()
rf_cv.fit(X_train, y_train)
rf_time = time.time() - start_time

print(f"Best params: {rf_cv.best_params_}")
print(f"Best CV RMSE: {-rf_cv.best_score_:.2f}")
print(f"Time: {rf_time:.2f}s")

rf_results = pd.DataFrame(rf_cv.cv_results_)[['param_max_features', 'param_min_samples_leaf', 'mean_test_score']]
rf_results['RMSE'] = -rf_results['mean_test_score']
print("\nRF Grid Search Results:")
print(rf_results.pivot(index='param_max_features', columns='param_min_samples_leaf', values='RMSE').round(2))

rf_train_pred = rf_cv.predict(X_train)
rf_train_rmse = np.sqrt(mean_squared_error(y_train, rf_train_pred))
rf_test_rmse = -rf_cv.best_score_
rf_r2 = r2_score(y_train, rf_train_pred)
n = len(y_train)
k = X_train.shape[1]
rf_bic = n * np.log(mean_squared_error(y_train, rf_train_pred)) + k * np.log(n)

In [ ]:
gbm = GradientBoostingRegressor(random_state=42)

gbm_tune_grid = {
    "n_estimators": [100, 200],
    "max_depth": [3, 5],
    "learning_rate": [0.05, 0.1],
    "min_samples_leaf": [10, 20]
}

gbm_cv = GridSearchCV(
    estimator=gbm,
    param_grid=gbm_tune_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=1,
    n_jobs=1
)

In [ ]:
start_time = time.time()
gbm_cv.fit(X_train, y_train)
gbm_time = time.time() - start_time

print(f"Best params: {gbm_cv.best_params_}")
print(f"Best CV RMSE: {-gbm_cv.best_score_:.2f}")
print(f"Time: {gbm_time:.2f}s")

gbm_train_pred = gbm_cv.predict(X_train)
gbm_train_rmse = np.sqrt(mean_squared_error(y_train, gbm_train_pred))
gbm_test_rmse = -gbm_cv.best_score_
gbm_r2 = r2_score(y_train, gbm_train_pred)
gbm_bic = n * np.log(mean_squared_error(y_train, gbm_train_pred)) + k * np.log(n)

In [ ]:
hgbm = HistGradientBoostingRegressor(random_state=42)

hgbm_tune_grid = {
    "max_iter": [100, 200],
    "max_depth": [5, 10],
    "learning_rate": [0.05, 0.1],
    "min_samples_leaf": [10, 20]
}

hgbm_cv = GridSearchCV(
    estimator=hgbm,
    param_grid=hgbm_tune_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=1,
    n_jobs=1
)

In [ ]:
start_time = time.time()
hgbm_cv.fit(X_train, y_train)
hgbm_time = time.time() - start_time

print(f"Best params: {hgbm_cv.best_params_}")
print(f"Best CV RMSE: {-hgbm_cv.best_score_:.2f}")
print(f"Time: {hgbm_time:.2f}s")

hgbm_train_pred = hgbm_cv.predict(X_train)
hgbm_train_rmse = np.sqrt(mean_squared_error(y_train, hgbm_train_pred))
hgbm_test_rmse = -hgbm_cv.best_score_
hgbm_r2 = r2_score(y_train, hgbm_train_pred)
hgbm_bic = n * np.log(mean_squared_error(y_train, hgbm_train_pred)) + k * np.log(n)

In [ ]:
rf_row = {
    'Model': 'Random Forest',
    'Coefficients': X_train.shape[1],
    'R-squared': round(rf_r2, 2),
    'BIC': int(rf_bic),
    'Training RMSE': round(rf_train_rmse, 2),
    'Test RMSE': round(rf_test_rmse, 2),
    'Time (s)': round(rf_time, 2)
}

gbm_row = {
    'Model': 'GBM',
    'Coefficients': X_train.shape[1],
    'R-squared': round(gbm_r2, 2),
    'BIC': int(gbm_bic),
    'Training RMSE': round(gbm_train_rmse, 2),
    'Test RMSE': round(gbm_test_rmse, 2),
    'Time (s)': round(gbm_time, 2)
}

hgbm_row = {
    'Model': 'HistGBM',
    'Coefficients': X_train.shape[1],
    'R-squared': round(hgbm_r2, 2),
    'BIC': int(hgbm_bic),
    'Training RMSE': round(hgbm_train_rmse, 2),
    'Test RMSE': round(hgbm_test_rmse, 2),
    'Time (s)': round(hgbm_time, 2)
}

In [ ]:
comparison = pd.concat([
    comparison,
    pd.DataFrame([rf_row, gbm_row, hgbm_row])
], ignore_index=True)

print("\n" + "="*60)
print("FINAL MODEL COMPARISON: ALL 5 MODELS")
print("="*60)
print(comparison.to_string(index=False))

comparison.to_csv(output + 'model_comparison_all.csv', index=False)

In [ ]:
best_idx = comparison['Test RMSE'].idxmin()
best_model = comparison.loc[best_idx]

print(f"\nBest model: {best_model['Model']}")
print(f"  - Test RMSE: €{best_model['Test RMSE']:.2f}")
print(f"  - Training RMSE: €{best_model['Training RMSE']:.2f}")
print(f"  - Time: {best_model['Time (s)']:.2f}s")

print("\nModel ranking by Test RMSE:")
ranking = comparison.sort_values('Test RMSE')[['Model', 'Test RMSE', 'Time (s)']]
print(ranking.to_string(index=False))

# Part 3: Feature Importance Analysis

In [ ]:
import matplotlib.ticker as mtick

rf_var_imp = pd.DataFrame({
    'variable': X_featnames,
    'imp': rf_cv.best_estimator_.feature_importances_
}).sort_values(by='imp', ascending=False).reset_index(drop=True)

rf_var_imp['cumulative_imp'] = rf_var_imp['imp'].cumsum()

gbm_var_imp = pd.DataFrame({
    'variable': X_featnames,
    'imp': gbm_cv.best_estimator_.feature_importances_
}).sort_values(by='imp', ascending=False).reset_index(drop=True)

gbm_var_imp['cumulative_imp'] = gbm_var_imp['imp'].cumsum()

print("Random Forest - Top 10 Features:")
display(rf_var_imp.head(10).style.format({'imp': '{:.1%}', 'cumulative_imp': '{:.1%}'}))
print("\nGradient Boosting - Top 10 Features:")
display(gbm_var_imp.head(10).style.format({'imp': '{:.1%}', 'cumulative_imp': '{:.1%}'}))

In [ ]:
cutoff = 0.01

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

rf_plot_data = rf_var_imp[rf_var_imp.imp > cutoff].sort_values(by='imp')
axes[0].barh(rf_plot_data['variable'], rf_plot_data['imp'], color='steelblue')
axes[0].set_xlabel('Importance')
axes[0].set_title('Random Forest: Feature Importances (>1%)')
axes[0].xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))
axes[0].grid(axis='x', alpha=0.3)

gbm_plot_data = gbm_var_imp[gbm_var_imp.imp > cutoff].sort_values(by='imp')
axes[1].barh(gbm_plot_data['variable'], gbm_plot_data['imp'], color='darkgreen')
axes[1].set_xlabel('Importance')
axes[1].set_title('Gradient Boosting: Feature Importances (>1%)')
axes[1].xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig(output + 'feature_importance_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
top10_comparison = pd.DataFrame({
    'Rank': range(1, 11),
    'RF Feature': rf_var_imp.head(10)['variable'].values,
    'RF Importance': rf_var_imp.head(10)['imp'].values,
    'GBM Feature': gbm_var_imp.head(10)['variable'].values,
    'GBM Importance': gbm_var_imp.head(10)['imp'].values
})

print("="*70)
print("TOP 10 FEATURES COMPARISON: Random Forest vs Gradient Boosting")
print("="*70)
display(top10_comparison.style.format({
    'RF Importance': '{:.1%}', 
    'GBM Importance': '{:.1%}'
}))

top10_comparison.to_csv(output + 'top10_features_comparison.csv', index=False)

In [ ]:
rf_top10 = set(rf_var_imp.head(10)['variable'])
gbm_top10 = set(gbm_var_imp.head(10)['variable'])
common_features = rf_top10.intersection(gbm_top10)
rf_only = rf_top10 - gbm_top10
gbm_only = gbm_top10 - rf_top10

print(f"Common features in top 10: {len(common_features)}")
print(f"  {sorted(common_features)}")
print(f"\nOnly in RF top 10: {len(rf_only)}")
print(f"  {sorted(rf_only)}")
print(f"\nOnly in GBM top 10: {len(gbm_only)}")
print(f"  {sorted(gbm_only)}")

In [ ]:
import shap

SAMPLE_SIZE = 1000
np.random.seed(42)
sample_idx = np.random.choice(X_train.shape[0], size=min(SAMPLE_SIZE, X_train.shape[0]), replace=False)
X_sample = X_train[sample_idx]

rf_explainer = shap.TreeExplainer(rf_cv.best_estimator_)
gbm_explainer = shap.TreeExplainer(gbm_cv.best_estimator_)

print("Computing SHAP values for Random Forest...")
rf_shap_values = rf_explainer.shap_values(X_sample)
print("Computing SHAP values for Gradient Boosting...")
gbm_shap_values = gbm_explainer.shap_values(X_sample)

In [ ]:
plt.figure(figsize=(8, 5))
shap.summary_plot(rf_shap_values, X_sample, feature_names=X_featnames, 
                  plot_type='bar', max_display=10, show=False)
plt.title('Random Forest: SHAP Feature Importance', fontsize=14)
plt.savefig(output + 'shap_rf.png', dpi=150, bbox_inches='tight')
plt.show()

plt.figure(figsize=(8, 5))
shap.summary_plot(gbm_shap_values, X_sample, feature_names=X_featnames, 
                  plot_type='bar', max_display=10, show=False)
plt.title('Gradient Boosting: SHAP Feature Importance', fontsize=14)
plt.savefig(output + 'shap_gbm.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
merged_imp = rf_var_imp[['variable', 'imp']].merge(
    gbm_var_imp[['variable', 'imp']], 
    on='variable', 
    suffixes=('_rf', '_gbm')
)
correlation = merged_imp['imp_rf'].corr(merged_imp['imp_gbm'])

rf_cumulative = rf_var_imp.head(10)['imp'].sum()
gbm_cumulative = gbm_var_imp.head(10)['imp'].sum()

print("="*70)
print("SUMMARY: Feature Importance Comparison")
print("="*70)
print(f"\n1. CORRELATION between RF and GBM importances: {correlation:.2f}")
print(f"\n2. TOP FEATURE:")
print(f"   - RF top feature: {rf_var_imp.iloc[0]['variable']} ({rf_var_imp.iloc[0]['imp']:.1%})")
print(f"   - GBM top feature: {gbm_var_imp.iloc[0]['variable']} ({gbm_var_imp.iloc[0]['imp']:.1%})")
print(f"\n3. OVERLAP: {len(common_features)}/10 features shared in both top 10 lists")
print(f"\n4. CONCENTRATION:")
print(f"   - RF: Top 10 features explain {rf_cumulative:.1%} of total importance")
print(f"   - GBM: Top 10 features explain {gbm_cumulative:.1%} of total importance")
if rf_only or gbm_only:
    print(f"\n5. KEY DIFFERENCES:")
    if rf_only:
        print(f"   - RF uniquely emphasizes: {', '.join(sorted(rf_only))}")
    if gbm_only:
        print(f"   - GBM uniquely emphasizes: {', '.join(sorted(gbm_only))}")

# Part II: Validity Analysis

Testing model performance on:
- **A. Later date:** Madrid Q2 2025 (temporal validity)
- **B. Other city:** Valencia Q1 2025 (external validity)

Note: Models are trained WITHOUT neighbourhood to enable cross-city comparison.

## Load All Three Datasets

In [ ]:
# Load all three cleaned datasets
data_madrid_q1 = pd.read_csv(data_out + "airbnb_madrid_25q1_clean.csv")
data_madrid_q2 = pd.read_csv(data_out + "airbnb_madrid_25q2_clean.csv")
data_valencia = pd.read_csv(data_out + "airbnb_valencia_25q1_clean.csv")

print("Dataset sizes:")
print(f"  Madrid Q1 2025: {data_madrid_q1.shape[0]:,} observations")
print(f"  Madrid Q2 2025: {data_madrid_q2.shape[0]:,} observations")
print(f"  Valencia Q1 2025: {data_valencia.shape[0]:,} observations")

# Check price distributions
print("\nPrice statistics:")
for name, df in [('Madrid Q1', data_madrid_q1), ('Madrid Q2', data_madrid_q2), ('Valencia', data_valencia)]:
    print(f"  {name}: mean=€{df['price'].mean():.0f}, median=€{df['price'].median():.0f}")

## Retrain Models on Madrid Q1 (Training Data)

In [ ]:
# Use full Madrid Q1 dataset for training (no holdout needed for validity)
train_data = data_madrid_q1.copy()

# Ensure amenities tuple is based on training data columns
amenities_valid = tuple([c for c in train_data.columns if c.startswith('d_') and 'host' not in c and 'instant' not in c])

# Full predictor set (no neighbourhood)
predictors_valid = basic_lev + basic_add + reviews + poly_lev + host_vars + amenities_valid

# Create design matrix for training
y_train_valid, X_train_valid = patsy.dmatrices("price ~ " + " + ".join(predictors_valid), train_data)
y_train_valid = np.ravel(y_train_valid)
X_featnames_valid = X_train_valid.design_info.column_names

print(f"Training features: {X_train_valid.shape[1]}")
print(f"Training observations: {X_train_valid.shape[0]}")

In [ ]:
# Train 5 models for validity testing
print("\n" + "="*60)
print("TRAINING MODELS FOR VALIDITY ANALYSIS")
print("="*60)

# 1. OLS (M7 - full model)
import statsmodels.formula.api as smf
ols_formula = "price ~ " + " + ".join(predictors_valid)
ols_model = smf.ols(ols_formula, data=train_data).fit()
print(f"OLS trained: {len(ols_model.params)} coefficients")

# 2. LASSO
scaler_valid = StandardScaler()
X_scaled = scaler_valid.fit_transform(X_train_valid)
lasso_model = LassoCV(alphas=np.arange(0.05, 1.01, 0.05), cv=5, random_state=42).fit(X_scaled, y_train_valid)
print(f"LASSO trained: {np.sum(lasso_model.coef_ != 0)} non-zero coefficients")

# 3. Random Forest (use best params from earlier)
rf_model = RandomForestRegressor(
    n_estimators=100, 
    max_features=rf_cv.best_params_['max_features'],
    min_samples_leaf=rf_cv.best_params_['min_samples_leaf'],
    random_state=42, 
    n_jobs=-1
).fit(X_train_valid, y_train_valid)
print(f"Random Forest trained")

# 4. GBM (use best params from earlier)
gbm_model = GradientBoostingRegressor(
    n_estimators=gbm_cv.best_params_['n_estimators'],
    max_depth=gbm_cv.best_params_['max_depth'],
    learning_rate=gbm_cv.best_params_['learning_rate'],
    min_samples_leaf=gbm_cv.best_params_['min_samples_leaf'],
    random_state=42
).fit(X_train_valid, y_train_valid)
print(f"GBM trained")

# 5. HistGBM (use best params from earlier)
hgbm_model = HistGradientBoostingRegressor(
    max_iter=hgbm_cv.best_params_['max_iter'],
    max_depth=hgbm_cv.best_params_['max_depth'],
    learning_rate=hgbm_cv.best_params_['learning_rate'],
    min_samples_leaf=hgbm_cv.best_params_['min_samples_leaf'],
    random_state=42
).fit(X_train_valid, y_train_valid)
print(f"HistGBM trained")

print("\nAll models trained successfully!")

## Predict and Evaluate on All Datasets

In [ ]:
def calculate_rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def evaluate_on_dataset(dataset, dataset_name, design_info):
    """Evaluate all 5 models on a dataset"""
    results = {'Dataset': dataset_name}
    
    # Create design matrix using the SAME design info from training
    y_test, X_test = patsy.dmatrices("price ~ " + " + ".join(predictors_valid), dataset)
    y_test = np.ravel(y_test)
    
    # OLS predictions
    ols_pred = ols_model.predict(dataset)
    results['OLS'] = calculate_rmse(y_test, ols_pred)
    
    # LASSO predictions (need to scale)
    X_test_scaled = scaler_valid.transform(X_test)
    lasso_pred = lasso_model.predict(X_test_scaled)
    results['LASSO'] = calculate_rmse(y_test, lasso_pred)
    
    # Tree model predictions
    results['RF'] = calculate_rmse(y_test, rf_model.predict(X_test))
    results['GBM'] = calculate_rmse(y_test, gbm_model.predict(X_test))
    results['HistGBM'] = calculate_rmse(y_test, hgbm_model.predict(X_test))
    
    return results

In [ ]:
# Evaluate on all three datasets
print("Evaluating models on all datasets...\n")

validity_results = []

# Madrid Q1 (training data - for reference)
validity_results.append(evaluate_on_dataset(data_madrid_q1, 'Madrid Q1 (train)', X_train_valid.design_info))

# Madrid Q2 (temporal validity)
validity_results.append(evaluate_on_dataset(data_madrid_q2, 'Madrid Q2 (later)', X_train_valid.design_info))

# Valencia (external validity)
validity_results.append(evaluate_on_dataset(data_valencia, 'Valencia Q1 (other city)', X_train_valid.design_info))

# Create results table
validity_df = pd.DataFrame(validity_results)
validity_df = validity_df.round(2)

print("="*80)
print("VALIDITY ANALYSIS: RMSE BY DATASET AND MODEL")
print("="*80)
print(validity_df.to_string(index=False))

# Save results
validity_df.to_csv(output + 'validity_comparison.csv', index=False)

In [ ]:
# Visualize validity results
fig, ax = plt.subplots(figsize=(12, 6))

validity_melted = validity_df.melt(id_vars='Dataset', var_name='Model', value_name='RMSE')

x = np.arange(len(validity_df))
width = 0.15
models = ['OLS', 'LASSO', 'RF', 'GBM', 'HistGBM']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

for i, model in enumerate(models):
    ax.bar(x + i*width, validity_df[model], width, label=model, color=colors[i])

ax.set_xlabel('Dataset')
ax.set_ylabel('RMSE (€)')
ax.set_title('Model Performance Across Datasets (Validity Analysis)')
ax.set_xticks(x + width * 2)
ax.set_xticklabels(validity_df['Dataset'], rotation=15, ha='right')
ax.legend(title='Model')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(output + 'validity_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## Validity Analysis Summary

In [ ]:
# Calculate RMSE increases
train_rmse = validity_df[validity_df['Dataset'] == 'Madrid Q1 (train)'].iloc[0]
q2_rmse = validity_df[validity_df['Dataset'] == 'Madrid Q2 (later)'].iloc[0]
valencia_rmse = validity_df[validity_df['Dataset'] == 'Valencia Q1 (other city)'].iloc[0]

print("="*80)
print("VALIDITY ANALYSIS SUMMARY")
print("="*80)

print("\n1. TEMPORAL VALIDITY (Madrid Q1 → Q2):")
for model in ['OLS', 'LASSO', 'RF', 'GBM', 'HistGBM']:
    pct_change = (q2_rmse[model] - train_rmse[model]) / train_rmse[model] * 100
    print(f"   {model}: €{train_rmse[model]:.2f} → €{q2_rmse[model]:.2f} ({pct_change:+.1f}%)")

print("\n2. EXTERNAL VALIDITY (Madrid → Valencia):")
for model in ['OLS', 'LASSO', 'RF', 'GBM', 'HistGBM']:
    pct_change = (valencia_rmse[model] - train_rmse[model]) / train_rmse[model] * 100
    print(f"   {model}: €{train_rmse[model]:.2f} → €{valencia_rmse[model]:.2f} ({pct_change:+.1f}%)")

# Find best model for each scenario
print("\n3. BEST MODEL BY DATASET:")
for _, row in validity_df.iterrows():
    model_rmses = row[['OLS', 'LASSO', 'RF', 'GBM', 'HistGBM']]
    best_model = model_rmses.idxmin()
    print(f"   {row['Dataset']}: {best_model} (RMSE = €{model_rmses[best_model]:.2f})")

print("\n4. KEY FINDINGS:")
avg_temporal = np.mean([(q2_rmse[m] - train_rmse[m]) / train_rmse[m] * 100 for m in models])
avg_external = np.mean([(valencia_rmse[m] - train_rmse[m]) / train_rmse[m] * 100 for m in models])
print(f"   - Average RMSE increase (temporal): {avg_temporal:+.1f}%")
print(f"   - Average RMSE increase (external): {avg_external:+.1f}%")
print(f"   - Models generalize {'well' if avg_external < 20 else 'with some degradation'} to Valencia")

## Discussion

**Temporal Validity (Madrid Q1 → Q2):**
- Models trained on Q1 data should perform similarly on Q2 data from the same city
- Small RMSE increases suggest stable market conditions
- Larger increases might indicate seasonal effects or market changes

**External Validity (Madrid → Valencia):**
- Testing on a different city evaluates whether the model captures generalizable relationships
- Price levels and market dynamics may differ between cities
- Excluding neighbourhood allows fair comparison (different neighbourhoods in each city)
- Some performance degradation is expected due to city-specific factors

**Model Robustness:**
- If tree-based models (RF, GBM) degrade more than linear models, they may be overfitting
- Simpler models often generalize better to new contexts
- The best model depends on the use case: prediction accuracy vs. generalizability